# Sumarised results from numerical experiments

In [ ]:
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
from signals import *
from plots import *
from samplers import *
from pylab import rcParams
rcParams['figure.figsize'] = 5, 2.5
from matplotlib.ticker import FormatStrFormatter, MultipleLocator

In [ ]:
signal = SignalExp([-0.3, 0.5, -0.1, 1])
change = 0.5*np.array([0, 1, 0, 0])
new_signal = SignalExp(signal.parameters + change)
alpha = np.linspace(0, 5)
t = np.linspace(0, 2*np.pi, 100)
sample_pos = np.multiply([0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7], np.pi)

fig, ax = plt.subplots(1)
ax.plot(t/np.pi, signal.get_samples(t), "C1", label="original")
sample_val = signal.get_samples(sample_pos)
ax.stem(sample_pos/np.pi, sample_val,  linefmt="C1", markerfmt="C1o")

new_pos = []
new_vals = []
for s in sample_pos:
    p = signal.path(s, change, n=10)
    new_pos.append(np.copy(p[len(p)-1]))
    
new_pos = np.array(new_pos)

ax.plot(t/np.pi, new_signal.get_samples(t), "--C0", label="perturbed")
ax.stem(new_pos/np.pi, new_signal.get_samples(new_pos), linefmt="--C0")
ax.axhline(0, c='k')
plt.ylabel(r"$f(x)$")
plt.legend()
ax.xaxis.set_major_formatter(FormatStrFormatter('%g $\pi$'))
ax.xaxis.set_major_locator(MultipleLocator(base=1.0))
plt.tight_layout()
fig.savefig('shifting.pdf')
plt.show()

In [ ]:
changes = np.array([1, -1.7, 0.7, 0])
changes = 3*changes

start_param = np.array([6, -10, 5, 0.5])
sample_values = [0.7,1.2,1.2,1.2]
polynomial = SignalPolynomial(start_param)

import itertools
from matplotlib import cm

pol = SignalPolynomial(start_param)
alpha = np.linspace(0,5)
colors = itertools.cycle(["b", "c", "g", "r"])
grays = itertools.cycle(['0.0','0.1','0.2','0.3','0.4'])
sample_positions = [0.2,0.4,0.6,0.8]


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10, 6))

ax = fig.add_subplot(111, projection='3d')
t = np.linspace(0, 1, 100)
pol = SignalPolynomial(start_param)
z = pol.get_samples(t)
ax.plot(t, np.zeros_like(t), z, 'k')

sample_values = pol.get_samples(sample_positions)

for s,v in zip(sample_positions, sample_values):
    color = next(colors)
    pol = SignalPolynomial(start_param)
    p = pol.path(s,0.1*changes)
    ax.plot(p,alpha,v*np.ones_like(alpha),lw=1,color=color)
    ax.scatter(p[0::10],alpha[0::10],v*np.ones(5),lw=1,color=color)
    
for i in range(5):
    pol2 = SignalPolynomial(start_param+i*changes)
    z = pol2.get_samples(t)
    ax.plot(t, np.ones_like(t)*i, z,next(grays))

ax.view_init(30, -100)
plt.show()

In [ ]:
param = np.pi/18

p = np.array([1.000000000000000000e+00,-2.825421682312351468e+00,-2.022133420929154990e-01,
              -1.483193386636804334e+00,-2.063111012904942587e+00,5.638963672080870015e-01,
              -3.154727063937948994e+00,1.396869864645395032e-01,-2.631363077020359764e+00,
              3.202233789725539292e+00])       
pol = SecondSurfacePolynomial(p)
t = np.linspace(-1,1, 100)
sampler = SurfaceSampler(pol, 2*len(p), [param, 1, 1], interval_length=2, sigma=0.0, beg=-1)

err1 = []
values = sampler.sample_values + 1e-2*np.random.randn(len(sampler.sample_values))
print(len(values))
angles = np.linspace(-np.pi/4.5, np.pi/4.5, 900)
for a in angles:
    err1.append(known_error(sampler.sample_positions, len(p), [a, 1, 1], values))

err = []
values = sampler.sample_values
angles = np.linspace(-np.pi/4.5, np.pi/4.5, 900)
for a in angles:
    err.append(known_error(sampler.sample_positions, len(p), [a, 1, 1], values))
    
fig = plt.figure()
plt.semilogy(np.degrees(angles), err)
# plt.semilogy(np.degrees(angles), err1)
plt.xlabel("angle (in degrees)")
plt.ylabel("cost function")
plt.title("no noise, pol. degree: 9, true angle: "+str(np.degrees(param)))
plt.grid()
plt.tight_layout()
plt.show()
fig.savefig("non_convex.pdf")

To generate experiment results, run for example:

`exec(open("surface-tests.py").read())`

This notebook only reads the results from files and generates plots.

In [ ]:
# listdir()

In [ ]:
# results = np.empty((2,4,10))
all_errors = np.zeros((4,11,100*13))

max_degree = 10

for n in range(3,max_degree):
    for nl in [0,1,2,3]:
        code1 = str(n)+"_1_"+str(nl)
        code2 = str(n)+"_1_"+"{0:.2f}".format(nl)
#         nsr = np.load('nsr_'+code+'.npy')
        try:
            errors = np.load('results/errors_'+code1+'.npy')
        except:
            errors = np.load('results/errors_'+code2+'.npy')
#         params = np.load('params_'+code+'.npy')
#         results[0,nl,n] = np.degrees(np.percentile(errors, q=50))
#         results[1,nl,n] = np.degrees(np.percentile(errors, q=95))
        errors = errors.flatten()
        all_errors[nl,n,:] = errors

all_errors = my_degrees(all_errors)
if np.isnan(all_errors).any():
    print("We have NaNs :(")
    print(str(np.count_nonzero(np.isnan(all_errors))))
all_errors[np.isnan(all_errors)]=20

In [ ]:
flierprops1 = dict(marker='+', markeredgecolor='purple')
medianprops1 = dict(linewidth=2.5, color='purple')
gp1 = dict(color='black')
flierprops2 = dict(marker='+', markeredgecolor='green')
medianprops2 = dict(linewidth=2.5, color='green')
gp2 = dict(color='gray', markeredgecolor='grayt')

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.boxplot(all_errors[0,2:max_degree-1,:].T, whis=[0, 100], flierprops=flierprops1, medianprops=medianprops1, boxprops=gp1, whiskerprops=gp1)
ax.boxplot(all_errors[2,2:max_degree-1,:].T, whis=[0, 100], flierprops=flierprops2, medianprops=medianprops2, boxprops=gp2, whiskerprops=gp2)
ax.set_yscale('log')
plt.xticks(range(1,max_degree-1),range(3,max_degree))
plt.xlabel("degree")
plt.ylabel("error")
plt.grid()
plt.tight_layout()
plt.show()
fig.savefig('degree.pdf')


In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
ind = np.arange(2,max_degree-1)
width = 0.4
rects1 = ax.bar(ind+(width/2), np.median(all_errors[3,3:max_degree,:],1), width, bottom=1e-14, color="C0", edgecolor='w', hatch="///")
rects2 = ax.bar(ind-(width/2), np.median(all_errors[0,3:max_degree,:],1), width, bottom=1e-14, color="C1")
# plt.bar(list(range(2,max_degree-1)), np.median(all_errors[0,3:max_degree,:],1), bottom=1e-14)
plt.xlabel("degree of polynomial")
plt.ylabel("median error")
plt.legend(["SNR 80dB", "no noise"], loc=6)
plt.grid()
plt.tight_layout()
plt.show()
fig.savefig('degree_bar.pdf')

In [ ]:
fig = plt.figure()
plt.hist(all_errors[0,5,:],  bins=np.logspace(-16, 2, 50), edgecolor='w')
print(np.count_nonzero(all_errors[0,5,:] > 1e-10)/len(all_errors[0,5,:]))
print(np.mean(all_errors[0,5,:]))
# plt.yscale('log', nonposy='clip')
plt.xscale('log', nonposx='clip')
plt.title("errors for pol. of degree 4")
plt.xlabel("error (in degrees)")
plt.grid()
plt.tight_layout()
plt.show()
fig.savefig('two_modes.pdf')

In [ ]:
a = np.linspace(-20,20, 13)

fig2, ax2 = plt.subplots()
angle_err = all_errors[0,3:10,:]
angle_err_med = np.median(angle_err.reshape((700, 13)),0)
ax2.semilogy(a, angle_err_med, 'C0--', label="no noise")
ax2.set_xlabel('angle')
plt.title("error dependence on the angle")

# ax3 = ax2.twinx()
ax3 = ax2
angle_err = all_errors[3,3:10,:]
angle_err_med = np.median(angle_err.reshape((700, 13)),0)
ax3.semilogy(a, angle_err_med, 'C1-', label='SNR ~ 80dB')
ax2.grid()
plt.legend()
plt.tight_layout()
fig2.savefig("angle.pdf")
    
# plt.plot(a, np.abs(a))
plt.show()

In [ ]:
results = np.empty((2,4,5,5))
all_errors = np.empty((4,5,5,100*13))
all_nsr = np.empty((4,5,5,100*13))

for (overs, idx) in zip([1,2,4,8], range(4)):
    for n in range(4,5):
        for nl in range(5):
            code = str(n)+"_"+str(overs)+"_"+str(nl)
            nsr = np.load('results/nsr_'+code+'.npy')
            errors = np.load('results/errors_'+code+'.npy')
    #         params = np.load('params_'+code+'.npy')
            errors[np.isnan(errors)]=1
            results[0,idx,nl,n] = np.degrees(np.percentile(errors, q=50))
            results[1,idx,nl,n] = np.degrees(np.mean(errors.flatten()))
            all_nsr[idx,nl,n,:] = nsr.flatten()
            all_errors[idx,nl,n,:] = errors.flatten()

In [ ]:
nse = 10.0**(-np.array(range(1,5)))
fig = plt.figure()
# plt.loglog(nse,results[0,:,1:,4].T, '--^')
styles = ['-.o','-o','--o',':o']
for idx in range(4):
    plt.loglog(nse, results[0,idx,1:,4].T, styles[idx])
# plt.semilogy(range(4),results[1,:,3:5], '^')
plt.xlabel("noise")
plt.ylabel("error")
plt.legend([r"no overs.",r"$2\times $ overs.",r"$4\times $ overs.", r"$8\times $ overs."], loc=2)
plt.grid()
plt.tight_layout()
plt.show()
fig.savefig('oversampling.pdf')

In [ ]:
n_pow = np.concatenate([np.arange(-2.46,-1.05, 0.07037), np.linspace(-1, 9, 199)])
n_len = len(n_pow)

results = np.empty((2,4,n_len))
all_errors = np.empty((4,n_len,100*13))
all_power = np.empty((4,n_len,100*13))
n = 5



for (overs, idx) in zip([1,2,4,8], range(4)):
    for (nl, n_idx) in zip(n_pow, range(len(n_pow))):
        code = str(n)+"_"+str(overs)+"_"+"{0:.2f}".format(nl)
        power = np.load('results/pow_'+code+'.npy')
        errors = np.load('results/errors_'+code+'.npy')
#         params = np.load('params_'+code+'.npy')
        errors[np.isnan(errors)]=1
        results[0,idx, n_idx] = np.degrees(np.percentile(errors, q=50))
        results[1,idx, n_idx] = np.degrees(np.mean(errors.flatten()))
        all_power[idx,n_idx,:] = power.flatten()
        all_errors[idx,n_idx,:] = errors.flatten()

In [ ]:
# calculate SNR:

snr = 10*(np.log(np.mean(all_power.flatten()))+2*n_pow)
beg = 10
fig = plt.figure()
# plt.loglog(nse,results[0,:,1:,4].T, '--^')
plt.semilogy(snr[beg:],results[0,0,beg:].T, "-.")
plt.semilogy(snr[beg:],results[1,0,beg:].T)
plt.semilogy(snr[beg:],results[0,3,beg:].T, "--")
plt.semilogy(snr[beg:],results[1,3,beg:].T, ":")
# plt.semilogy(range(4),results[1,:,3:5], '^')
plt.xlabel(r"$SNR_{dB}$")
plt.ylabel("error (in degrres)")
plt.legend([r"median error (no overs.)",r"mean error (no overs.)", r"median error 8$\times$ overs.",r"mean error 8$\times$ overs."], loc='best')
plt.grid()
plt.tight_layout()
plt.show()
fig.savefig('snr2.pdf')